# **Data Cleaning**

## Objectives

* We will evaluate missing data
* Clean the data

## Inputs

* outputs/datasets/collection/house_prices_records.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned

## CRISP-DM

* "Data Preparation"

## Conclusions 
The following data cleaning steps were taken:
* Arbitrary Number Imputer – ['2ndFlrSF', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']
* Categorical Variable Imputer – ['BsmtFinType1', 'GarageFinish']
* Mean Median Imputer – ['BedroomAbvGr' , 'GarageYrBlt', 'LotFrontage',]


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Collected Data

Load the data generated from the previous notebook

In [ ]:
import pandas as pd
df_raw_path = "outputs/datasets/collection/house_prices_records.csv"
df = pd.read_csv(df_raw_path)
df.head(5)

---

# Data Exploration 

We will explore Data Cleaning. We are interested in checking the distribution and shape from a variable with missing data

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

We now know from the above that there is missing data within the above variables data. We will extract more information about the types of data that are missing below. 

In [ ]:
df.filter(vars_with_missing_data).info()

We will now gnerate a profile report. This will give us more details.

In [ ]:
from pandas_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

# Correlation and PPS analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

def heatmap_corr(df,threshold, figsize=(20,12), font_annot = 8):
  if len(df.columns) > 1:
    mask = np.zeros_like(df, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    mask[abs(df) < threshold] = True

    fig, axes = plt.subplots(figsize=figsize)
    sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                linewidth=0.5
                     )
    axes.set_yticklabels(df.columns, rotation = 0)
    plt.ylim(len(df.columns),0)
    plt.show()


def heatmap_pps(df,threshold, figsize=(20,12), font_annot = 8):
    if len(df.columns) > 1:

      mask = np.zeros_like(df, dtype=np.bool)
      mask[abs(df) < threshold] = True

      fig, ax = plt.subplots(figsize=figsize)
      ax = sns.heatmap(df, annot=True, xticklabels=True,yticklabels=True,
                       mask=mask,cmap='rocket_r', annot_kws={"size": font_annot},
                       linewidth=0.05,linecolor='grey')
      
      plt.ylim(len(df.columns),0)
      plt.show()



def CalculateCorrAndPPS(df):
  df_corr_spearman = df.corr(method="spearman")
  df_corr_pearson = df.corr(method="pearson")

  pps_matrix_raw = pps.matrix(df)
  pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

  pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
  print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
  print(pps_score_stats.round(3))

  return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix,CorrThreshold,PPS_Threshold,
                      figsize=(20,12), font_annot=8 ):

  print("\n")
  print("* Analyze how the target variable for your ML models are correlated with other variables (features and target)")
  print("* Analyze multi colinearity, that is, how the features are correlated among themselves")

  print("\n")
  print("*** Heatmap: Spearman Correlation ***")
  print("It evaluates monotonic relationship \n")
  heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Pearson Correlation ***")
  print("It evaluates the linear relationship between two continuous variables \n")
  heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Power Predictive Score (PPS) ***")
  print(f"PPS detects linear or non-linear relationships between two columns.\n"
        f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
  heatmap_pps(df=pps_matrix,threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

Calculate Correlations and Power Predictive Score

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

In [ ]:
%matplotlib inline
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.4, PPS_Threshold =0.2,
                  figsize=(12,10), font_annot=10)

# Data Cleaning

Explore and assess missing data levels 

Below we will display the missing data levels again, this time in a dataframe.

In [ ]:
def EvaluateMissingData(df):
  missing_data_absolute = df.isnull().sum()
  missing_data_percentage = round(missing_data_absolute/len(df)*100 , 2)
  df_missing_data = (pd.DataFrame(
                          data= {"RowsWithMissingData": missing_data_absolute,
                                 "PercentageOfDataset": missing_data_percentage,
                                 "DataType":df.dtypes}
                                  )
                    .sort_values(by=['PercentageOfDataset'],ascending=False)
                    .query("PercentageOfDataset > 0")
                    )

  return df_missing_data

In [ ]:
EvaluateMissingData(df)

From the above table it is evident that there is some substantial missing data. We need to investigate each variable's data to determine how we intend to move forward with it. Having data that is simply missing is not acceptable, so we can either drop the entire variable or impute an alternative into the missing fields. The actions we take are dependent on the conclusions we draw upon observing the data and how we interpret it. I will be using the Pandas profiling report from the Sale_Price_Study notebook for my insights. 


#### Most common value: ArbitraryNumberImputer
The below 2 variables have over 80% missing data. Usually this can be a justification for dropping the data variable from the study. However, first we have to rule out whether an assumption can be made about what the potential missing value could be and what action needs to be taken. 
- `EnclosedPorch` - The values that are present in the data account for 7.9% of the entire data and the values are Zero. In comparison the the remaining data that is present, zero is by far more prominent. Therefore it would be acceptable to assume that the missing values should also be "0" (zero). 
- `WoodDeckSF` - As with EnclosedPorch, we have a similar picture, this time 5.3% of the data present is made up of "0", whereas 89.38% of the data is missing. Again we would be right to assume that "0" should be imputed into fields where a value is missing. 
The zero in both of the above variables represents the absence of the variable in the property. 
- `2ndFlrSF` - I estimate that this variable is referring to the square footage of the second floor of the proeprty. 5.8% of the data is missing. Again we are going to opt to use the most common value to fill in the blanks. 
- `MasVnrArea` - 0.5% will be batched with "zero" which is again the most common choice.

#### Most likely value: CategoricalVariableImputer
- `GarageFinish` - There are 4 valid categories, unfinshed, finished, RFn and None. It would be fair to assume that the missing data most likely should be put under the "none" category. 
- `BsmtFinType` - Here we have 7 categories, 6 of which refrence a particular acronym for a type of finishing. The 7th category is "none" and it would again be best practice to put the 7.8% mising data under "none". 

#### Taking a median: MeanMedianImputer
`AbvGr` is an abbreviation for "Above Grade". This is a term used by American Realtors, that refrences when either in whole or in part, the area being refrenced is above the surface of the ground. With the below variable, this wold translate to "bedroom above the surface of the ground". 
- `BedroomAbvGr` - The histogram reflects a normal distribution. Therefore we can take median value for the 6.7% that is void of data. 
- `LotFrontage` - 17.74% of the data is missing. Having inpected the Histogram, the distribution looks normal. Therefore we should use the median of the data to fill in the missing data. 
 - `GarageYrBlt` - As the data inputs refrence actual years, putting the data under the most common value would skew the data as the data is missing and wouldnt fit under any particular category that exists. Instead we need to use the median method here to blend the missing data into the existing data. As it is a small percentage at 5.5%, the data will not be skewed as a result. 

 We need to clean the data according to our above observations now. 

# Data cleaning spreadsheet summary

The first step to cleaning the data is adding the function of DataCleaningEffect() taken from Code Institute's Feature Engine module. This code will help assess the effect of cleaning the data in the following instances when:
- imput mean, median or arbitrary number is a numerical variable
- replace with 'Missing' or most frequent a categorical variable

We will set parameters:
- df_original (unclean data)
- df_cleaned (data that has been cleaned as per the assigned variables)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

Now we will start adding the code to clean the data in the following order:
1. ArbritraryNumberImputer
2. CategoricalVariableImputer
3. MeanMedianImputer

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer

variables_method = ['2ndFlrSF', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']
variables_method

imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)

df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

In [ ]:
from feature_engine.imputation import CategoricalImputer
variables_method = ['BsmtFinType1', 'GarageFinish']
variables_method

imputer = CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=variables_method)

df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                  df_cleaned=df_method,
                  variables_applied_with_method=variables_method)

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_method = ['BedroomAbvGr' , 'GarageYrBlt', 'LotFrontage',]
variables_method

imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)

df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                  df_cleaned=df_method,
                  variables_applied_with_method=variables_method)

# Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

## Impute changes to Variables 

In summary; we will now make the chamges we tested above to the data. 
Arbit
cate
meanmedian 

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer
variables_method = ['2ndFlrSF', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']
imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)
imputer.fit_transform(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
from feature_engine.imputation import CategoricalImputer
variables_method = ['BsmtFinType1', 'GarageFinish']
imputer = CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=variables_method)

imputer.fit_transform(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
from feature_engine.imputation import MeanMedianImputer
variables_method = ['BedroomAbvGr' , 'GarageYrBlt', 'LotFrontage',]
imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)

imputer.fit_transform(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

In [ ]:
df_missing_data = EvaluateMissingData(TestSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

## TrainSet

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

## TestSet

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)